# Basic Classification: Herkunft vorhersagen

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Make numpy printouts easier to read.
np.set_printoptions(precision=3, suppress=True)

In [ ]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import categorical_crossentropy

## Dateset
### Daten herunterladen und in ein Dataframe speichern

In [ ]:
url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data'
column_names = ['MPG', 'Cylinders', 'Displacement', 'Horsepower', 'Weight',
                'Acceleration', 'Model Year', 'Origin']

raw_dataset = pd.read_csv(url, names=column_names,
                          na_values='?', comment='\t',
                          sep=' ', skipinitialspace=True)

In [ ]:
dataset = raw_dataset.copy()
dataset.head()

### Daten säubern

Zeilen mit leeren Felder löschen.

In [ ]:
dataset.isna().sum()

In [ ]:
dataset = dataset.dropna()

Die `"Origin"` Spalte ist eigentlich categorical und nicht Numerisch. Dies müssen wir umbauen.


In [ ]:
dataset['Origin'] = dataset['Origin'].map({1: 'USA', 2: 'Europe', 3: 'Japan'})

In [ ]:
dataset = pd.get_dummies(dataset, prefix='', prefix_sep='')
dataset.tail()

### Aufteilen der Daten in Train und Test
Nun teilen wir den Datensatz in einen Trainingssatz und einen Testsatz auf.

Wir werden den Testsatz bei der Endauswertung unserer Modelle verwenden.

In [ ]:
train_dataset = dataset.sample(frac=0.8, random_state=0)
test_dataset = dataset.drop(train_dataset.index)

### Features von Labels trennen

Trennen Sie den Zielwert, das "Label", von den Features. Dieses Label ist der Wert, auf dessen Vorhersage Sie das Modell trainieren werden.

In [ ]:
train_features = train_dataset.copy()
test_features = test_dataset.copy()

train_labels = train_features[['USA', 'Europe', 'Japan']].copy()
test_labels = test_features[['USA', 'Europe', 'Japan']].copy()

# Länder aus dem Training/Test features entfernen
train_features = train_features.drop(['USA', 'Europe', 'Japan'], axis=1)
test_features = test_features.drop(['USA', 'Europe', 'Japan'], axis=1)

In [ ]:
print(f'Shape of train data: {train_features.shape}')
print(f'Shape of test data: {test_features.shape}')
print(f'Shape of train labels: {train_labels.shape}')
print(f'Shape of test labels: {test_labels.shape}')

In [ ]:
train_features.head()

In [ ]:
train_labels.head()

## Lineare Klassifikation

### Eine Variable


Das Training eines Modells mit `tf.keras` beginnt typischerweise mit der Definition der Modellarchitektur.

In diesem Fall verwenden Sie ein Sequentielles Modell. Dieses Modell stellt eine Abfolge von Schritten dar. In diesem Fall gibt es zwei Schritte:

* Normalisierung der eingegebenen `horsepower`.
* Wenden Sie eine lineare Transformation ($y = mx+b$) an, um 3 Ausgaben unter Verwendung von `layers.Dense` zu erzeugen.
* Erzeugen Sie mithilfe der Softmax Funktion die Wahrscheinlichkeiten für den Ursprung des Autos

Zuerst den horsepower `Normalization` layer:

In [ ]:
horsepower = np.array(train_features['Horsepower'])

horsepower_normalizer = preprocessing.Normalization(input_shape=[1,])
horsepower_normalizer.adapt(horsepower)

### Model

In [ ]:
horsepower_model = Sequential()
horsepower_model.add(horsepower_normalizer)
horsepower_model.add(Dense(units=3, activation='softmax'))

horsepower_model.summary()

Dieses Modell wird `Origin` aus `Horsepower` vorhersagen.

Lassen Sie das untrainierte Modell mit den ersten 10 Pferdestärkenwerten laufen. Das Ergebnis wird nicht gut sein, aber Sie werden sehen, dass es die erwartete Form `(10,3)` hat:

In [ ]:
horsepower_model.predict(horsepower[:10])

### Model trainieren

In [ ]:
opt_adam = Adam(learning_rate=0.1)
horsepower_model.compile(
    optimizer=opt_adam,
    loss=categorical_crossentropy,
    metrics=['categorical_accuracy'])

Sobald das Training konfiguriert ist, verwenden Sie Model.fit(), um das Training auszuführen:

In [ ]:
history = horsepower_model.fit(
    train_features['Horsepower'], 
    train_labels,
    epochs=100,
    validation_split = 0.2)

### Evaluation - Wie ist das Training verlaufen?

In [ ]:
def plot_loss(history):
  plt.plot(history.history['loss'], label='loss')
  plt.plot(history.history['val_loss'], label='val_loss')
  plt.xlim([-1, 10])
  plt.xlabel('Epoch')
  plt.ylabel('Error [Origin]')
  plt.legend()
  plt.grid(True)

In [ ]:
def plot_accuracy(history):
  plt.plot(history.history['categorical_accuracy'], label='accuracy')
  plt.plot(history.history['val_categorical_accuracy'], label='val_accuracy')
  plt.xlim([-1, 10])
  plt.xlabel('Epoch')
  plt.ylabel('Accuracy [Origin]')
  plt.legend()
  plt.grid(True)

Da es sich um eine Regression mit einer einzigen Variable handelt, ist es einfach, die Vorhersagen des Modells als Funktion des Inputs zu betrachten: